Take two DCBOE files and find differences based on the hashes. 

* old_dcboe = the CSV in the current head commit
* new_dcboe = the CSV in the current working directory, not yet committed

In [1]:
import io
import os
import pandas as pd
from git import Repo

os.chdir('../..') # root of repo is two directories above this notebook

In [2]:
csv_file_path = 'data/dcboe/candidates_dcboe.csv'

oa_repo = Repo('.') 
commit = oa_repo.head.commit
targetfile = commit.tree / csv_file_path

with io.BytesIO(targetfile.data_stream.read()) as f:
    old_dcboe = pd.read_table(f, sep=',', encoding='utf-8')

In [3]:
new_dcboe = pd.read_csv(csv_file_path)

In [4]:
len(old_dcboe)

420

In [5]:
len(new_dcboe)

420

## Old hashes not in new file

In [6]:
old_hashes_not_in_new = ~( old_dcboe['dcboe_hash_id'].isin(new_dcboe['dcboe_hash_id']))
old_dcboe[old_hashes_not_in_new]

,dcboe_hash_id,smd_id,candidate_name,pickup_date,filed_date
132,c9ccac3b4ddd343b3d7212e7d1fc8dab5f4fbe55f81440...,smd_3F01,Noah Bell,2020-08-04,2020-08-05
399,9bd61943b16542005aa92a0ab6066a18e51a92b76bc8e8...,smd_8D01,Patricia Carmon,2020-07-17,2020-08-05


## New hashes not in old file

In [7]:
new_hashes_not_in_old = ~( new_dcboe['dcboe_hash_id'].isin(old_dcboe['dcboe_hash_id']))
new_dcboe[new_hashes_not_in_old]

,dcboe_hash_id,smd_id,candidate_name,pickup_date,filed_date
175,0f7d13376694cff37980e2fefef317b04bcd82f033076c...,smd_4C03,Cecelia Waldeck,2020-07-31,2020-08-05
374,a18cc1ecaaf81f445eeb1e80ddc7d01168aa3d278ea46f...,smd_8A06,Kristina Leszczak,2020-08-26,NaN


## Same hash, changed info

Have any fields changed on the same hash? 

In [8]:
df = pd.merge(old_dcboe, new_dcboe, how='inner', on='dcboe_hash_id', suffixes=['_old', '_new'])

In [9]:
columns_to_check = [c for c in old_dcboe.columns if c != 'dcboe_hash_id']

for c in columns_to_check:
    num_differences = sum(df[c + '_old'] != df[c + '_new'])
    if num_differences > 0:
        print(df[df[c + '_old'] != df[c + '_new']])